# Dataset Info.

# train.csv [파일]

     PRODUCT_ID : 제품의 고유 ID

     Y_Class : 제품 품질 상태(Target) 

     0 : 적정 기준 미달 (부적합)

     1 : 적합

     2 : 적정 기준 초과 (부적합)

     Y_Quality : 제품 품질 관련 정량적 수치

     TIMESTAMP : 제품이 공정에 들어간 시각

     LINE : 제품이 들어간 공정 LINE 종류 ('T050304', 'T050307', 'T100304', 'T100306', 'T010306', 'T010305' 존재)

     PRODUCT_CODE : 제품의 CODE 번호 ('A_31', 'T_31', 'O_31' 존재)

     X_1 ~ X_2875 : 공정 과정에서 추출되어 비식별화된 변수


## test.csv [파일]

    PRODUCT_ID : 제품의 고유 ID

    TIMESTAMP : 제품이 공정에 들어간 시각

    LINE : 제품이 들어간 공정 LINE 종류 ('T050304', 'T050307', 'T100304', 'T100306', 'T010306', 'T010305' 존재)

    PRODUCT_CODE : 제품의 CODE 번호 ('A_31', 'T_31', 'O_31' 존재)X_1 ~ X_2875 : 공정 과정에서 추출되어 비식별화된 변수


## sample_submission.csv [파일] - 제출 양식

    PRODUCT_ID : 제품의 고유 ID

    Y_Class : 예측한 제품 품질 상태

    0 : 적정 기준 미달 (부적합)

    1 : 적합

    2 : 적정 기준 초과 (부적합)


# 실제 공정 과정에서의 데이터로, 보안상의 이유로 일부 변수가 비식별화 처리 되었습니다. (X변수)
# 'LINE', 'PRODUCT_CODE'는 Train / Test 모두 동일한 종류가 존재합니다.

In [16]:
import random
import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, normalize
from sklearn.ensemble import RandomForestClassifier

In [17]:
data = pd.read_csv('open/train.csv')
test_df = pd.read_csv('open/test.csv')
Y_Q = data['Y_Quality']
#Y_Quality : 약 0.5~0.58 => normalization 필요
Y_C = data['Y_Class']

train_y = data['Y_Class']
train_x = data.drop(columns=['PRODUCT_ID','TIMESTAMP', 'Y_Class', 'Y_Quality'])
X_ = []


for i in range(1,2876):
    tmp = 'X_'+str(i)
    if len(train_x[tmp].unique()) >20:
        pass
    else:
        X_.append(tmp)




test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])



In [18]:
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

In [19]:
train_x.describe()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
count,598.000000,598.00000,598.0,598.000000,598.000000,598.0,598.000000,598.000000,598.000000,598.000000,...,598.000000,598.000000,598.000000,598.000000,598.000000,598.000000,598.0,598.0,598.0,598.0
mean,1.406355,55.51505,0.0,26.262542,6.065217,0.0,28.481605,5.864548,24.202341,1.167224,...,8.496204,8.964498,8.295351,8.638763,11.145435,0.165552,0.0,0.0,0.0,0.0
std,4.655332,47.03581,0.0,22.201747,5.140970,0.0,24.308159,4.960485,21.978767,0.986744,...,19.189554,20.294317,18.755287,19.632753,24.961071,0.371989,0.0,0.0,0.0,0.0
min,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,2.000000,91.00000,0.0,45.000000,10.000000,0.0,45.000000,10.000000,31.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,2.000000,95.00000,0.0,45.000000,10.000000,0.0,50.000000,10.000000,52.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,103.000000,102.00000,0.0,45.000000,11.000000,0.0,62.000000,11.000000,52.000000,2.000000,...,60.240000,68.660000,60.410000,59.930000,79.750000,1.000000,0.0,0.0,0.0,0.0


In [20]:
test_x.describe()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
count,310.000000,310.000000,310.0,310.000000,310.000000,310.0,310.000000,310.000000,310.000000,310.000000,...,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000
mean,3.400000,74.035484,0.0,35.274194,8.051613,0.0,38.006452,7.848387,33.241935,1.567742,...,5.710516,5.814516,5.483258,5.944774,6.998097,0.122581,7.199677,7.312903,7.063548,6.643871
std,14.820217,39.103820,0.0,18.552095,4.252992,0.0,20.260279,4.128939,19.791866,0.824538,...,16.320561,16.656275,15.775015,17.001694,19.980751,0.328485,41.904735,42.482780,41.312700,38.612285
min,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,87.250000,0.0,45.000000,10.000000,0.0,45.000000,10.000000,31.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,93.000000,0.0,45.000000,10.000000,0.0,45.000000,10.000000,31.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.000000,96.000000,0.0,45.000000,10.000000,0.0,50.000000,10.000000,52.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,154.000000,102.000000,0.0,45.000000,11.000000,0.0,67.000000,11.000000,52.000000,2.000000,...,58.060000,60.490000,60.310000,58.290000,67.240000,1.000000,268.000000,278.600000,271.400000,245.600000


In [21]:
print(max(Y_Q))
print(min(Y_Q))
print(Y_Q.describe())

0.57884127
0.500855556
count    598.000000
mean       0.530896
std        0.007401
min        0.500856
25%        0.527535
50%        0.530436
75%        0.533433
max        0.578841
Name: Y_Quality, dtype: float64


In [22]:
print(Y_C.value_counts())


1    407
2    103
0     88
Name: Y_Class, dtype: int64


In [23]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

In [24]:
RF = RandomForestClassifier(random_state=37).fit(train_x, train_y)
print('Done.')

ValueError: could not convert string to float: 'T050304'

In [ ]:
preds = RF.predict(test_x)
print('Done.')

Done.


In [ ]:
submit = pd.read_csv('open/sample_submission.csv')

In [ ]:
print(len(submit['Y_Class']))
print(len(preds))

310
310


In [ ]:
submit['Y_Class'] = preds

In [ ]:
submit.to_csv('./baseline_submission.csv', index=False)

In [ ]:
print(len(preds))

310
